In [34]:
#%pip install openpyxl
#%pip install pandas

#!pip install openpyxl pandas
!pip install plotly

!pip install "jupyterlab>=3" "ipywidgets>=7.6"

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.4/139.4 KB 155.1 kB/s eta 0:00:00 kB/s eta 0:00:01:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.9/214.9 KB 920.5 kB/s eta 0:00:00 kB/s eta 0:00:01:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 5.0 MB/s eta 0:00:00m eta 0:00:010:00:01


In [12]:

path = r"../../data/deg/Teff_DoublevsTeff_AAVS1_deg.csv"
path



'../../data/deglists/csv/Teff_DoublevsTeff_AAVS1_deg.csv'

In [13]:
import pandas as pd

data_df = pd.read_csv(path)
data_df

KeyError: 'baseMean'

In [62]:
import numpy as np

data_df = data_df.loc[:, ['gene_id', 'log2FoldChange', 'pvalue', 'padj', 'gene_name']]
data_df["log10pvalue"] = -np.log10(data_df['pvalue']) 
data_df["log10pvalue"] = -np.log10(data_df['pvalue'])

data_df["type"] = np.where(data_df['pvalue'] >= 0.05, "not diff", np.where(data_df['log2FoldChange'] > 0, "UP", "DOWN"))

data_df

,gene_id,log2FoldChange,pvalue,padj,gene_name,log10pvalue,type
0,ENSG00000108702,-3.035035,9.378426e-11,0.000002,CCL1,10.027870,DOWN
1,ENSG00000099985,-1.911403,1.244014e-07,0.001079,OSM,6.905175,DOWN
2,ENSG00000164399,-2.374052,1.829456e-06,0.007168,IL3,5.737678,DOWN
3,ENSG00000145850,-1.828880,1.928530e-06,0.007168,TIMD4,5.714773,DOWN
4,ENSG00000180730,-2.609514,2.202688e-06,0.007168,SHISA2,5.657047,DOWN
...,...,...,...,...,...,...,...
32489,ENSG00000204644,-0.497462,NaN,1.000000,ZFP57,NaN,DOWN
32490,ENSG00000226279,-4.046552,NaN,1.000000,RPL12P10,NaN,DOWN
32491,ENSG00000147862,-7.892086,NaN,1.000000,NFIB,NaN,DOWN
32492,ENSG00000185630,-3.448552,NaN,1.000000,PBX1,NaN,DOWN


In [1]:
import plotly.express as px
import pandas as pd
import numpy as np

def create_volcano(input_path, output_path):
    
    data_df = pd.read_csv(input_path)

    # filter my baseMean
    data_df = data_df.loc[data_df["baseMean"] > 15]

    # filter by if padj is NA
    # data_df.dropna(subset=['padj'], inplace=True)
    
    data_df = data_df.loc[:, ['gene_id', 'log2FoldChange', 'pvalue', 'padj', 'gene_name']]
    data_df["log10padj"] = -np.log10(data_df['padj']) 
    data_df["log10padj"] = -np.log10(data_df['padj'])
    data_df["type"] = np.where(data_df['padj'] >= 0.05, "not diff", np.where(data_df['log2FoldChange'] > 0, "UP", "DOWN"))
    
    fig = px.scatter(data_df, x="log2FoldChange", y="log10padj", color="type", 
                     custom_data=['gene_id', 'gene_name', 'log2FoldChange', 'pvalue', 'padj'],
                     color_discrete_map={
                        "UP": "#EF553B",      # red
                        "DOWN": "#636EFA",    # blue
                        "not diff": "#00CC96" # green
                     })
    
    fig.update_traces(mode="markers")
    
    fig.update_traces(
        hovertemplate="<br>".join([
            "gene_name: %{customdata[1]}",
            "log2fold: %{customdata[2]}",
            "pvalue: %{customdata[3]}",
            "padj: %{customdata[4]}",
            "gene_id: %{customdata[0]}",
        ])
    )
    fig.write_html(output_path)
    #return fig


In [2]:
paths = [
    r"../../data/deg/Teff_S_vs_A_both3.csv",
    r"../../data/deg/Treg_S_vs_A_both3.csv",
]

for path in paths:
    create_volcano(path, "../../results/" + path.split('/')[-1][:-3] + "html")



/home/elizabeth/.local/lib/python3.10/site-packages/plotly/express/_core.py:2065: FutureWarning: When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.
  sf: grouped.get_group(s if len(s) > 1 else s[0])
/home/elizabeth/.local/lib/python3.10/site-packages/plotly/express/_core.py:2065: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.

